# Import Statements

In [1]:
#suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
#import statements
from bs4 import BeautifulSoup
import time
import html2text
import re
import os
import pandas as pd
import unicodedata
import glob
import itertools
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
%run scrape.py #functions: simple_get, is_good_response, log_error

# Character

In [3]:
#create dictionary for character html
wtnv_html_dict_char = dict()
main_url = 'https://nightvale.fandom.com'

In [4]:
#pull character list
char_url = 'https://nightvale.fandom.com/wiki/Category:Characters'

tz = simple_get(char_url)
time.sleep(1)

#parse html
html = BeautifulSoup(tz, 'html.parser')

In [5]:
#get links
links = html.findAll("a", {"class" : "category-page__member-link"})

for link in links:
    #get page
    tz_l = simple_get(main_url+link['href'])
    time.sleep(1)
    #parse html
    html_l = BeautifulSoup(tz_l, 'html.parser')
    #follow every episode link
    #pull release date, episode num?
    name = html_l.find("meta", {"property" : "og:title"}).get("content")
    wtnv_html_dict_char[name] = html_l

In [6]:
#format all character names
ner_char = []
for entry in wtnv_html_dict_char:
    entry = entry.strip()
    if 'Category' not in entry:
        div = wtnv_html_dict_char[entry].findAll("div", {"data-source" : "aka"})
        if len(div) > 0:
            akas = str(div[0].find("div", {"class" : "pi-data-value pi-font"}))[35:-6]
            sp = ['Chad Bowinger', 'Glow Cloud (ALL HAIL)', 'Janice', 'Old Woman Josie', 
                  'The Man in the Tan Jacket (character)', 'The Faceless Old Woman (character)', 
                  'John Peters (you know, the farmer?)']
            if entry not in sp:
                #deal with the two
                en = entry.strip()+"::CHAR:"
                if len(entry.split(' ')) > 1:
                    en = en+entry.strip()[0]+entry.strip().split(' ')[-1]
                else:
                    en = en+entry.strip()
                ner_char.append(en)
                en2 = akas.strip()+"::CHR:"
                if len(entry.strip().split(' ')) > 1:
                    en2 = en2+entry.strip()[0]+entry.strip().split(' ')[-1]
                else:
                    en2 = en2+entry.strip()
                ner_char.append(en2)
            else:
                brack = ['Glow Cloud (ALL HAIL)','John Peters (you know, the farmer?)', 
                         'The Faceless Old Woman (character)', 'The Man in the Tan Jacket (character)']
                if entry in brack:
                    ent = entry.strip().replace('(character)','').replace('(ALL HAIL)','').replace('(you know, the farmer?)','').strip()
                    ner_char.append(ent+"::CHR:"+ent[0]+ent.split(' ')[-1])
                    for aka in akas.replace('know,',':').split(","):
                        ner_char.append(aka.replace('(All Hail)','').replace('(you : the farmer?)','').strip()+"::CHR:"+ent[0]+ent.split(' ')[-1])
                else:
                    #print(entry)
                    l1 = akas.split('<br/>')
                    if len(l1) > 1:
                        ner_char.append(entry.strip()+"::CHR:"+entry.strip()[0]+entry.strip().split(' ')[-1])
                        for l in l1:
                            ner_char.append(l.strip()+"::CHR:"+entry.strip()[0]+entry.strip().split(' ')[-1])
                    else:
                        l2 = akas.split(',')
                        if len(l2) > 1:
                            ner_char.append(entry.strip()+"::CHR:"+entry.strip()[0]+entry.strip().split(' ')[-1])
                            for l in l2:
                                ner_char.append(l.strip()+"::CHR:"+entry.strip()[0]+entry.strip().split(' ')[-1])
                        else:
                            ner_char.append(entry.strip()+"::CHR:"+entry.strip())
                            ner_char.append(akas.split('by')[0].strip()+"::CHR:"+entry.strip())

        else:
            en = entry.strip()+"::CHR:"
            if len(entry.strip().split(' ')) > 1:
                en = en+entry.strip()[0]+entry.strip().split(' ')[-1]
            else:
                en = en+entry.strip()
            ner_char.append(en)

# Creatures

In [8]:
#scrape list of creatures
wtnv_html_dict_crea = dict()
creature_url = 'https://nightvale.fandom.com/wiki/Category:Night_Vale_creatures'

tz = simple_get(creature_url)
time.sleep(1)

#parse html
html = BeautifulSoup(tz, 'html.parser')

In [9]:
#get links
links = html.findAll("a", {"class" : "category-page__member-link"})

for link in links:
    #get page
    tz_l = simple_get(main_url+link['href'])
    time.sleep(1)
    #parse html
    html_l = BeautifulSoup(tz_l, 'html.parser')
    
    name = html_l.find("meta", {"property" : "og:title"}).get("content")
    wtnv_html_dict_crea[name] = html_l

In [10]:
ner_crea = []
for entry in wtnv_html_dict_crea:
    entry = entry.replace('(creature)','').strip()
    if 'List' not in entry:
        if len(entry.strip().split(' ')) > 1:
            ner_crea.append(entry+"::CrAn:"+entry[0]+entry.split(' ')[-1])
        else:
            ner_crea.append(entry+"::CrAn:"+entry)

# Locations

In [11]:
wtnv_html_dict_loc = dict()
loc_url = 'https://nightvale.fandom.com/wiki/Category:Locations'

tz = simple_get(loc_url)
time.sleep(1)

#parse html
html = BeautifulSoup(tz, 'html.parser')

In [12]:
#get links
links = html.findAll("a", {"class" : "category-page__member-link"})

for link in links:
    #get page
    tz_l = simple_get(main_url+link['href'])
    time.sleep(1)
    #parse html
    html_l = BeautifulSoup(tz_l, 'html.parser')
    
    name = html_l.find("meta", {"property" : "og:title"}).get("content")
    wtnv_html_dict_loc[name] = html_l

In [13]:
ner_loc = []
for entry in wtnv_html_dict_loc:
    entry = entry.replace('(town)','').strip()
    if (entry[-1] != '2') & ('ocations' not in entry):
        ner_loc.append(entry+"::LOC:"+entry)

# Orgs

In [14]:
wtnv_html_dict_org = dict()
org_urls = ['https://nightvale.fandom.com/wiki/Category:Organization', 
            'https://nightvale.fandom.com/wiki/Category:Night_Vale_organizations']

tz = simple_get(org_urls[0])
time.sleep(1)

#parse html
html = BeautifulSoup(tz, 'html.parser')

In [15]:
#get links
links = html.findAll("a", {"class" : "category-page__member-link"})

for link in links:
    #get page
    tz_l = simple_get(main_url+link['href'])
    time.sleep(1)
    #parse html
    html_l = BeautifulSoup(tz_l, 'html.parser')
    
    name = html_l.find("meta", {"property" : "og:title"}).get("content")
    wtnv_html_dict_org[name] = html_l

In [16]:
tz = simple_get(org_urls[1])
time.sleep(1)

#parse html
html = BeautifulSoup(tz, 'html.parser')

In [17]:
#get links
links = html.findAll("a", {"class" : "category-page__member-link"})

for link in links:
    #get page
    tz_l = simple_get(main_url+link['href'])
    time.sleep(1)
    #parse html
    html_l = BeautifulSoup(tz_l, 'html.parser')
    
    name = html_l.find("meta", {"property" : "og:title"}).get("content")
    wtnv_html_dict_org[name] = html_l

In [18]:
ner_org = []
for entry in wtnv_html_dict_org:
    ner_org.append(entry+"::ORG:"+entry)

# Write to txt File

In [19]:
li = ner_loc+ner_org+ner_char+ner_crea

In [20]:
with open('wtnv_ner.txt', 'w') as f:
    for item in li:
        f.write("%s\n" % item)